# Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Компания провела опрос и попросила нас подготовить дашборд с его итогами.

**Задачи проекта:**

1) используя SQL-запрос нам необходимо выгрузить данные из таблиц;

2) на основе выгруженных данных сформировать дашборд;

3) с помощью дашборда ответить на вопросы заказчика:

- Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников?

[Дашборд](https://public.tableau.com/views/projectinwork_16918391100470/Prefabricatedprojectyandex_practicum?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

[Презентация](https://disk.yandex.ru/i/DlzW3_9HNLmaDg)

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine # импортируем библиотеки

In [2]:
path_to_db = 'telecomm_csi.db' # путь к файлу
engine = create_engine(f'sqlite:///{path_to_db}', echo = False) 

In [3]:
# сформируем sql запрос
query = """
SELECT user_id,
       lt_day,
       case 
           when lt_day <= 365 then 'да'
           when lt_day > 365 then 'нет'
       end as is_new,
       age,
       case
           when gender_segment = 1 then 'женщина'
           when gender_segment = 0 then 'мужчина'
           else 'неизвестный'
       end as gender_segment,
       os_name,
       cpe_type_name,
       country,
       city,
       a.title as age_segment,
       ts.title as traffic_segment,
       ls.title as lifetime_segment,
       nps_score,
       case
           when nps_score >= 9 then 'cторонник'
           when nps_score = 8 OR nps_score = 7 then 'нейтральный'
           when nps_score <= 6 then 'критик'
       end as nps_group
FROM user AS us
INNER JOIN location AS l ON us.location_id = l.location_id
INNER JOIN age_segment AS a ON us.age_gr_id = a.age_gr_id
INNER JOIN traffic_segment AS ts ON us.tr_gr_id = ts.tr_gr_id
INNER JOIN lifetime_segment AS ls ON us.lt_gr_id = ls.lt_gr_id
"""

df = pd.read_sql(query, engine) # создадим переменную с нашей таблицей

In [4]:
df.sample(10) # посмотрим случайные 10 строк

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
483051,YZO7HU,1202,нет,46.0,женщина,ANDROID,SMARTPHONE,Россия,Казань,05 45-54,04 1-5,08 36+,2,критик
366531,SYGOAH,2673,нет,44.0,женщина,ANDROID,SMARTPHONE,Россия,Пенза,04 35-44,05 5-10,08 36+,10,cторонник
332403,R77S82,1393,нет,22.0,женщина,IOS,SMARTPHONE,Россия,Екатеринбург,02 16-24,24 100+,08 36+,10,cторонник
57834,CZ4Y07,1089,нет,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Казань,04 35-44,06 10-15,08 36+,8,нейтральный
1325,A2H7EL,4595,нет,50.0,мужчина,ANDROID,SMARTPHONE,Россия,Екатеринбург,05 45-54,04 1-5,08 36+,8,нейтральный
81832,E7V2SP,273,да,28.0,женщина,ANDROID,SMARTPHONE,Россия,Калининград,03 25-34,08 20-25,05 7-12,7,нейтральный
57701,CYVCMT,161,да,25.0,женщина,ANDROID,SMARTPHONE,Россия,НижнийНовгород,03 25-34,07 15-20,04 4-6,10,cторонник
122138,GAI7B3,5697,нет,34.0,женщина,ANDROID,SMARTPHONE,Россия,Барнаул,03 25-34,04 1-5,08 36+,10,cторонник
143905,HFARFH,4632,нет,29.0,женщина,ANDROID,SMARTPHONE,Россия,Екатеринбург,03 25-34,04 1-5,08 36+,3,критик
419889,VPYRXO,409,нет,41.0,женщина,ANDROID,SMARTPHONE,Россия,Красноярск,04 35-44,06 10-15,06 13-24,10,cторонник
